In [1]:
import math
import decimal
import time
import scipy
import scipy.sparse as scsp
from scipy import linalg
import numpy as np

In [2]:
class Oracle_Chebyshev_Oscillator():
    def __init__(self, n):
        self.n = n

    def F(self, x):
        F = np.zeros(self.n)
        F[0] = x[0] - 1
        for i in range(self.n-1):
            
            F[i+1] = x[i+1] - 2 * x[i] ** 2 + 1
        return F

    def f1(self, x):
        return np.sqrt(np.sum(self.F(x) ** 2))

    def hat_f1(self, x):
        return self.f1(x) / np.sqrt(self.n)

    def jacobian_F(self, x):
        J = np.zeros((self.n, self.n))
        J[0,0] = 1
        for i in range(self.n-1):
            J[i+1,i+1] = 1
            J[i+1,i] = - 4 * x[i]
        return J

    def grad_hat_f1(self, x):
        g = np.zeros(self.n)
        g[0] = 2 * (x[0] - 1) - 8 * x[0] * (x[1] - 2 * x[0] ** 2 + 1)
        for i in range(self.n-2):
            g[i+1] = 2 * (x[i+1] - 2 * x[i] ** 2 + 1) - 8 * x[i+1] * (x[i+2] - 2 * x[i+1] ** 2 + 1)

        g[self.n-1] = 2 * (x[self.n-1] - 2 * x[self.n-2] ** 2 + 1)
        return g / (2 * self.n * self.hat_f1(x))



In [3]:
class Oracle_Trigonometric_Oscillator():
    def __init__(self, n):
        self.n = n

    def F(self, x):
        F = np.zeros(self.n)
        F[0] = x[0] - 1
        for i in range(self.n-1):
            F[i+1] = x[i+1] + np.cos(np.pi * x[i])
        return F

    def f1(self, x):
        return np.sqrt(np.sum(self.F(x) ** 2))

    def hat_f1(self, x):
        return self.f1(x) / np.sqrt(self.n)

    def jacobian_F(self, x):
        J = np.zeros((self.n, self.n))
        J[0,0] = 1
        for i in range(self.n-1):
            J[i+1,i+1] = 1
            J[i+1,i] = - np.pi * np.sin(np.pi * x[i-1])
        return J

    def grad_hat_f1(self, x):
        g = np.zeros(self.n)
        g[0] = 2 * (x[0] - 1) - 2 * np.pi * np.sin(np.pi * x[0]) * (x[1] + np.cos(np.pi * x[0]))
        for i in range(self.n-2):
            g[i+1] = 2 * (x[i+1] + np.cos(np.pi * x[i]))  - 2 * np.pi * np.sin(np.pi * x[i+1]) * (x[i+2] + np.cos(np.pi * x[i+1]))

        g[self.n - 1] = 2 * (x[self.n-1] + np.cos(np.pi * x[self.n-2]))
        return g / (2 * self.n * self.hat_f1(x))



In [4]:
def GKL_bidiagonalization(A):
    A = np.matrix(A)
    m,n = A.shape
    
    p = max(m, n)
    alpha = np.zeros(p)
    beta = np.zeros(p-1)
    U = np.zeros((p, m))  # in fact it's U.H
    V = np.zeros((p, n))  # in fact it's V.H
    # Transposed matrices are used for easier slicing
    U[0, 0] = 1
    
    for i in range(p):
        V[i] = A.H@U[i] - beta[i-1]*V[i-1]
        alpha[i] = np.linalg.norm(V[i])
        V[i] /= alpha[i]

        if i > p - 2: continue
        U[i+1] = A@V[i] - alpha[i]*U[i]
        beta[i] = np.linalg.norm(U[i+1])
        U[i+1] /= beta[i]
    U,V = map(np.matrix, (U, V))
    B = scsp.diags((alpha, beta), [0, -1]).toarray()    
    return U.H, B, V


In [5]:
A = np.array([[1, 2, 3],
         [4, 5, 2],
         [5, 2, 9]])
U, B, V = GKL_bidiagonalization(A)
print(np.array(B))

[[ 3.74165739  0.          0.        ]
 [11.00649159  3.77910984  0.        ]
 [ 0.          0.59163378  4.38468256]]


In [9]:
def  FullGaussNewton(x_, oracle):
    start_time1 = time.time()
    iter_max = 1000000
    iter_counter = 0
    oracle_counter = 0
    func_res = []
    p = oracle.n
    grad_mapping = []
    I_p = np.identity(p)
    I_n = np.identity(oracle.n)
    L = 1
    eps = 10 ** -5
    x_new = x_
    x_curr = x_
    func_res.append(oracle.hat_f1(x_new))

    while ((iter_counter <= iter_max) and (np.linalg.norm(x_new - x_curr, ord=2) > eps)) or (iter_counter == 0): 
#        print(x_curr)
        x_curr = x_new.copy()
        J_curr = oracle.jacobian_F(x_curr)
        G_curr = J_curr.T
        U_curr, Lambda_curr, V_curr = GKL_bidiagonalization(G_curr)
        Lambda_curr = np.array(Lambda_curr)
        V_curr = np.array(V_curr)
        U_curr = np.array(U_curr)
        g_curr = oracle.grad_hat_f1(x_curr)
        f_curr = oracle.hat_f1(x_curr)
        gamma_curr = 1 / (p * f_curr)
        while True:
            C_curr = I_p + (gamma_curr / L) * Lambda_curr @ Lambda_curr.T
#            print(C_curr)
            d_curr = U_curr.T @ g_curr
            x_tmp_curr = TDMAsolver(C_curr, d_curr)
            x_new = x_curr - (U_curr @ x_tmp_curr) / L
#            print(x_curr)
            f_new = oracle.hat_f1(x_new)
            oracle_counter = oracle_counter + 1

            if (f_new <= f_curr
                  + g_curr @ (x_new - x_curr)
                  + 1 / (2 * p * f_curr) * np.linalg.norm(G_curr.T @ (x_new - x_curr), ord=2) ** 2
                  + L / 2 * np.linalg.norm(x_new - x_curr, ord=2) ** 2):
                 
                func_res.append(f_new)
                grad_mapping.append(np.linalg.norm(x_new - x_curr, ord=2))    
                iter_counter = iter_counter + 1
#                print('iter = ', iter_counter, np.linalg.norm(x_new - x_curr, ord=2))
                L = L / 2

                break
        
            else:
                L = 2 * L
        
    start_time2 = time.time()
#   print(iter_counter, '.......', oracle_counter, '.......', min(grad_mapping),
#         '.......', min(func_res))
#   print(x_new)
    return iter_counter, oracle_counter, min(grad_mapping), min(func_res), (start_time2 - start_time1)

In [10]:
n = 9
x_0 = np.ones(n)
x_0[0] = -1
oracle = Oracle_Trigonometric_Oscillator(n)
repeats = 1
iter_counter = np.zeros(repeats)
oracle_counter = np.zeros(repeats)
times = np.zeros(repeats)
mapping = np.zeros(repeats)
func = np.zeros(repeats)
for i in range(repeats):
    iter_counter[i], oracle_counter[i], mapping[i], func[i], times[i] = FullGaussNewton(x_0, oracle)

    

print('iter_counter = ', np.mean(iter_counter))
print('oracle_counter = ', np.mean(oracle_counter))
print('grad_mapping = ', np.mean(mapping))
print('func = ', np.mean(func))
print('time = ', np.mean(times))

KeyboardInterrupt: 

In [11]:
def MomentumStepExtrapolation(oracle, x_prev, x_curr):
    tau_prev = 0
    tau_curr = 1
    y_prev = x_prev + tau_prev * (x_curr - x_prev)
    y_curr = x_prev + tau_curr * (x_curr - x_prev)
    while (oracle.grad_hat_f1(y_curr) @ (x_curr - x_prev) < 0 and oracle.hat_f1(y_curr) <= oracle.hat_f1(y_prev)):
        y_prev = y_curr.copy()
        tau_curr = tau_curr * 2
        y_curr = x_prev + tau_curr * (x_curr - x_prev)

    return y_curr

In [12]:
def MomentumStepArmijo(oracle, x_prev, x_curr):
    alpha = 1 / 3
    beta = 2 / 3
    tau = 1
    phi = oracle.hat_f1(x_prev)
    grad_phi = oracle.grad_hat_f1(x_prev) @ (x_curr - x_prev)
    y = x_prev + tau * (x_curr - x_prev)
    if (((phi + beta * grad_phi * tau) <= oracle.hat_f1(y)) and ((phi + alpha * grad_phi * tau) >= oracle.hat_f1(y))):
        return y
    if (phi + beta * grad_phi * tau > oracle.hat_f1(y)):
        tau_1 = tau
        tau_2 = 2 * tau_1
        while True:
            y1 = x_prev + tau_1 * (x_curr - x_prev)
            y2 = x_prev + tau_2 * (x_curr - x_prev)
            if (phi + beta * grad_phi * tau_1 > oracle.hat_f1(y1)) and (phi + alpha * grad_phi * tau_2 < oracle.hat_f1(y2)):
                break
            if (phi + beta * grad_phi * tau_1 <= oracle.hat_f1(y1)) and (phi + alpha * grad_phi * tau_1 >= oracle.hat_f1(y1)):
                return y1
            tau_1 = tau_2
            tau_2 = tau_1 * 2
    else:
        tau_2 = tau
        tau_1 = tau_2 / 2
        while True:
            y1 = x_prev + tau_1 * (x_curr - x_prev)
            y2 = x_prev + tau_2 * (x_curr - x_prev)
            if (phi + beta * grad_phi * tau_1 > oracle.hat_f1(y1)) and (phi + alpha * grad_phi * tau_2 < oracle.hat_f1(y2)):
                break
            if (phi + beta * grad_phi * tau_1 <= oracle.hat_f1(y1)) and (phi + alpha * grad_phi * tau_1 >= oracle.hat_f1(y1)):
                return y1
            tau_2 = tau_1
            tau_1 = tau_2 / 2
#    print('4')
    y1 = x_prev + tau_1 * (x_curr - x_prev)
    y2 = x_prev + tau_2 * (x_curr - x_prev)
    if (tau_2 < 10-3):
        return y2
    while True:
#        print('4')
        hat_tau = (tau_1 + tau_2) / 2
        y_hat = x_prev + hat_tau * (x_curr - x_prev)
        if (phi + alpha * grad_phi * hat_tau < oracle.hat_f1(y_hat)):
            tau_2 = hat_tau
        elif (phi + beta * grad_phi * hat_tau > oracle.hat_f1(y_hat)):
            tau_1 = hat_tau
        else: 
            return y_hat
#    print('7')
#    return x_curr + tau_1 * (x_curr - x_prev)



In [23]:
def  FullGaussNewtonDoubling(x, oracle):
    start_time1 = time.time()
    p = oracle.n
    I_n = np.identity(oracle.n)
    iter_max = 100000
    iter_counter = 0
    oracle_counter = 0
    func_res = []
    grad_mapping = []
    I_p = np.identity(p)
    L = 1
    k = 0
    eps = 10 ** -5
    x_curr = x
    func_res.append(oracle.hat_f1(x_curr))

    J_curr = oracle.jacobian_F(x_curr)
    G_curr = J_curr.T
    U_curr, Lambda_curr, V_curr = GKL_bidiagonalization(G_curr)
    Lambda_curr = np.array(Lambda_curr)
    V_curr = np.array(V_curr)
    U_curr = np.array(U_curr)
    g_curr = oracle.grad_hat_f1(x_curr)
    f_curr = oracle.hat_f1(x_curr)
    gamma_curr = 1 / (p * f_curr)
    while True:
        C_curr = I_p + (gamma_curr / L) * Lambda_curr @ Lambda_curr.T
        d_curr = U_curr.T @ g_curr
        x_tmp_curr = np.array(linalg.solve(C_curr, d_curr))
        x_new = x_curr - (U_curr @ x_tmp_curr) / L
        f_new = oracle.hat_f1(x_new)
        oracle_counter = oracle_counter + 1

        if (f_new <= f_curr
            + g_curr @ (x_new - x_curr)
            + 1 / (2 * p * f_curr) * np.linalg.norm(G_curr.T @ (x_new - x_curr), ord=2) ** 2
            + L / 2 * np.linalg.norm(x_new - x_curr, ord=2) ** 2):
            grad_mapping.append(np.linalg.norm(x_new - x_curr, ord=2))
            y_curr = x_new
            break
        else:
            L = L * 2
    while True:
        iter_counter = iter_counter + 1
        x_prev = x_curr.copy()
        L = L / 2
        J_curr = oracle.jacobian_F(y_curr)
        G_curr = J_curr.T
        U_curr, Lambda_curr, V_curr = GKL_bidiagonalization(G_curr)
        Lambda_curr = np.array(Lambda_curr)
        V_curr = np.array(V_curr)
        U_curr = np.array(U_curr)
        g_curr = oracle.grad_hat_f1(y_curr)
        f_curr = oracle.hat_f1(y_curr)
        gamma_curr = 1 / (p * f_curr)
        while True:
            C_curr = I_p + (gamma_curr / L) * Lambda_curr @ Lambda_curr.T
            d_curr = U_curr.T @ g_curr
            x_tmp_curr = np.array(linalg.solve(C_curr, d_curr))
            x_new = y_curr - (U_curr @ x_tmp_curr) / L
            f_new = oracle.hat_f1(x_new)
            oracle_counter = oracle_counter + 1

            if (f_new <= f_curr
                + g_curr @ (x_new - x_curr)
                + 1 / (2 * p * f_curr) * np.linalg.norm(G_curr.T @ (x_new - x_curr), ord=2) ** 2
                + L / 2 * np.linalg.norm(x_new - x_curr, ord=2) ** 2):
                 
                x_curr = x_new.copy()
                L = L / 2
#                print('yes')
                break
            else:
#                print('no')
                L = 2 * L

#        print(iter_counter, '.......', np.linalg.norm(x_new - y_curr, ord=2), '.......', oracle.hat_f1(x_new))
        func_res.append(oracle.hat_f1(x_curr))
        grad_mapping.append(np.linalg.norm(x_curr - y_curr, ord=2))
        if ((np.linalg.norm(x_new - y_curr, ord=2) <= eps) or (iter_counter == iter_max)):
            break
#        print('mom')
#        print('x_curr', x_curr)
#        print('x_prev', x_prev)
    #    print('1')
        y_curr = MomentumStepArmijo(oracle, x_prev, x_curr)
    #    print('2')
        print(iter_counter, '.......')
#   #     print('y_curr', y_curr)
  #  print(iter_counter, '.......', oracle_counter, '.......', np.min(np.array(grad_mapping)), '.......', np.min(np.array(func_res)))
    start_time2 = time.time()
    return iter_counter, oracle_counter, min(grad_mapping), min(func_res), (start_time2 - start_time1) 

In [ ]:
n = 3
x_0 = np.ones(n)
x_0[0] = -1
oracle = Oracle_Chebyshev_Oscillator(n)
repeats = 1
iter_counter = np.zeros(repeats)
oracle_counter = np.zeros(repeats)
times = np.zeros(repeats)
mapping = np.zeros(repeats)
func = np.zeros(repeats)
for i in range(repeats):
    iter_counter[i], oracle_counter[i], mapping[i], func[i], times[i] = FullGaussNewtonDoubling(x_0, oracle)

    

print('iter_counter = ', np.mean(iter_counter))
print('oracle_counter = ', np.mean(oracle_counter))
print('grad_mapping = ', np.mean(mapping))
print('func = ', np.mean(func))
print('time = ', np.mean(times))

In [8]:
def TDMAsolver(A, d):
    '''
    TDMA solver, a b c d can be NumPy array type or Python list type.
    refer to http://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm
    and to http://www.cfd-online.com/Wiki/Tridiagonal_matrix_algorithm_-_TDMA_(Thomas_algorithm)
    '''
    a=[]
    b=[]
    c=[]
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
                if i == j:
                    b.append(A[i][j])
                elif i+1==j:
                    c.append(A[i][j])
                elif i==j+1:
                    a.append(A[i][j])
    nf = len(d) # number of equations
    ac, bc, cc, dc = map(np.array, (a, b, c, d)) # copy arrays
    for it in range(1, nf):
        mc = ac[it-1]/bc[it-1]
        bc[it] = bc[it] - mc*cc[it-1] 
        dc[it] = dc[it] - mc*dc[it-1]
   
    xc = bc
    xc[-1] = dc[-1]/bc[-1]

    for il in range(nf-2, -1, -1):
        xc[il] = (dc[il]-cc[il]*xc[il+1])/bc[il]

    return xc




def get_tridiag(A):
    a=[]
    b=[]
    c=[]
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
                if i == j:
                    b.append(A[i][j])
                elif i+1==j:
                    c.append(A[i][j])
                elif i==j+1:
                    a.append(A[i][j])
    return a,b,c

